<a href="https://colab.research.google.com/github/seenu-g/Experiments/blob/master/py_examples/numpy/minst/train_test_minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import sys
%load_ext autoreload
%autoreload 2
drive.mount('/content/gdrive',force_remount=True)

In [0]:
import torch
import torchvision

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

import sys
import os
folder = '/content/gdrive/My Drive/Github_Experiments/'
os.chdir(folder)
print(os.getcwd()) 

In [0]:
batch_size_train = 64
log_interval = 10
batch_size_test = 1000
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [0]:
import matplotlib.pyplot as plt
def show_images(example_data) :
  fig = plt.figure()
  for i in range(20):
    plt.subplot(5,4,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    #plt.imshow(example_data[i][0].numpy().squeeze(), cmap='gray_r');
    plt.title("Ground Truth: {}".format(example_targets[i]))
    plt.xticks([])
    plt.yticks([])

In [0]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_labels) = next(examples)
print(example_data.shape)
print(batch_idx)
print(example_labels.shape)
show_images(example_data)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
#defines the way we compute our output using the given layers and functions
#One can print out tensors somewhere in the forward pass for easier debugging.
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=-1)

In [0]:
learning_rate = 0.01
momentum = 0.5
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)
print(network)


show initial weight and initial gradient - experiment Not Required for running this program

In [0]:
examples = enumerate(train_loader)
batch_idx, (train_example_data, train_example_label) = next(train_examples)
print('Initial weights - ', network.conv1.weight.shape)  
print('Initial weights - ', network.conv2.weight.shape)  
print('Initial weights - ', network.fc1.weight.shape)  
print('Initial weights - ', network.fc2.weight.shape)  

#print('Initial weights - ', network.conv1.weight)  
optimizer.zero_grad()
print('Initial Gradient -shape', network.conv1.weight.grad.shape)
print('Initial Gradient -shape', network.conv2.weight.grad.shape)
print('Initial Gradient -shape', network.fc1.weight.shape)  
print('Initial Gradient -shape', network.fc2.weight.shape)  

#print('Initial Gradient -', network.fc1.weight.grad) # this always has been zero matrix
#print('Initial Gradient -', network.fc1.weight) # this always has been initialized to random values

show initial weight and initial gradient - experiment # Not Required for running this program

In [0]:
output = network(train_example_data)
loss = F.nll_loss(output, train_example_label)
loss.backward() 
print('UpdatedGradient shape-', network.conv1.weight.grad.shape)
print('UpdatedGradient -', network.conv1.weight.grad)
optimizer.step()
#print('Updated weights - ', network.conv1.weight)

In [0]:
n_epochs = 3
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)#compute a negative log-likelihodd loss between the output and the ground truth label
    loss.backward() # collect a new set of gradients which we propagate back into each of the network's parameters using optimizer.step()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      #save and load their internal state
      torch.save(network.state_dict(), '/content/gdrive/My Drive/Github_Experiments/' +'model.pth')
      torch.save(optimizer.state_dict(), '/content/gdrive/My Drive/Github_Experiments/' +'optimizer.pth')
  print("\nTraining Time (in minutes) =",(time()-time0)/60)

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad(): # avoid storing the computations done producing the output of our network in the computation graph.
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [0]:
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

In [0]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
#plt.plot(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

In [0]:
continued_network = Net()
continued_optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)
# load the internal state of the network and optimizer when we last saved them.
network_state_dict = torch.load('/content/gdrive/My Drive/Github_Experiments/' +'model.pth')
continued_network.load_state_dict(network_state_dict)

optimizer_state_dict = torch.load('/content/gdrive/My Drive/Github_Experiments/' +'optimizer.pth')
continued_optimizer.load_state_dict(optimizer_state_dict)

for i in range(n_epochs + 1,9):
  test_counter.append(i*len(train_loader.dataset))
  train(i)
  test()

In [0]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
#plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

This can be done everytime train and test are done. Then use output to predict and display actuals

In [0]:
with torch.no_grad():
  output = network(example_data)

In [0]:
fig = plt.figure()
for i in range(20):
  plt.subplot(5,4,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])